### **Tensorflow4강 Model Restore**

training시킨다는 것은 함수를 만드는 과정이다.

drop out은 트레이닝의 속도, 최적화되는 weight의 범위를 넓히기 위함이다.

**batch normalization**
![ex4_1](./image/4_1.png)
학습을 할 때, sigmooid의 역함수(기울기)가 계속 곱해지는데, 기울기가 0과 비슷한 지점으로 가까워 지면 결국 w = w - 0 이랑 같은 소리다.   
기울기가 존재하는 부분에 위치하기 위해 보정을 해주는 단계가 **batch normalization** 이다.   
training이 되지않는다!를 해결하는 방법

**regularization**
![ex4_2](./image/4_2.png)
overfitting을 막기 위해서 람다를 조절해주는 방법이다.   
training이 잘 되지 않는다!를 해결하는 방법

In [1]:
import tensorflow as tf

In [2]:
input_data = [[1, 5, 3, 7, 8, 10, 12],
              [5, 8, 10, 3, 9, 7, 1]]
label_data = [[0, 0, 0, 1, 0],
              [1, 0, 0, 0, 0]]

In [3]:
INPUT_SIZE = 7
HIDDEN1_SIZE = 10
HIDDEN2_SIZE = 8
CLASSES = 5
Learning_Rate = 0.5

In [4]:
x = tf.placeholder( tf.float32, shape=[None, INPUT_SIZE], name='x')
y_ = tf.placeholder( tf.float32, shape=[None, CLASSES], name='y_')

In [5]:
tensor_map = {x:input_data, y_:label_data}

In [6]:
w_h1 = tf.Variable(tf.truncated_normal(shape=[INPUT_SIZE, HIDDEN1_SIZE]), name='w_h1')
b_h1 = tf.Variable(tf.zeros([HIDDEN1_SIZE]), dtype=tf.float32, name='b_h1')

w_h2 = tf.Variable(tf.truncated_normal(shape=[HIDDEN1_SIZE, HIDDEN2_SIZE]), name='w_h2')
b_h2 = tf.Variable(tf.zeros([HIDDEN2_SIZE]), dtype=tf.float32, name='b_h2')

w_o = tf.Variable(tf.truncated_normal(shape=[HIDDEN2_SIZE, CLASSES]), name='w_o')
b_o = tf.Variable(tf.zeros([CLASSES]), dtype=tf.float32, name='b_o')

In [7]:
param_list = [w_h1, b_h1, w_h2, b_h2, w_o, b_o]
saver = tf.train.Saver(param_list)

In [8]:
hidden1 = tf.sigmoid(tf.matmul(x, w_h1) + b_h1, name='hidden1')
hidden2 = tf.sigmoid(tf.matmul(hidden1, w_h2) + b_h2, name='hieen2')
y = tf.sigmoid(tf.matmul(hidden2, w_o) + b_o, name='y')

In [9]:
sess = tf.Session()

In [10]:
saver.restore(sess, "./checkpoint/my_model.ckpt")
result = sess.run(y, tensor_map)
print(result)

INFO:tensorflow:Restoring parameters from ./checkpoint/my_model.ckpt
[[0.01343621 0.0040397  0.00262886 0.98461354 0.00354611]
 [0.98818123 0.00143028 0.00281574 0.01243097 0.00199468]]


initialization을 check point로 하게되면 이미 학습된 데이터를 다시 학습시킬 수 있다.

In [11]:
cost = tf.reduce_mean(tf.reduce_sum(-y_*tf.log(y)-(1-y_)*tf.log(1-y), reduction_indices=1))
train = tf.train.GradientDescentOptimizer(Learning_Rate).minimize(cost)

accuray = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1)), tf.float32))

saver.restore(sess, "./checkpoint/my_model.ckpt")

for i in range(100):
    _, loss, acc = sess.run([train, cost, accuray], tensor_map)
    
    if i % 10 == 0:
        saver.save(sess, "./checkpoint/my_model.ckpt")
        print("----------------------------")
        print("step : ", i)
        print("loss : ", loss)
        print("accuarcy : ", acc)
        print("result : ", result)

sess.close()

INFO:tensorflow:Restoring parameters from ./checkpoint/my_model.ckpt
----------------------------
step :  0
loss :  0.034955837
accuarcy :  1.0
result :  [[0.01343621 0.0040397  0.00262886 0.98461354 0.00354611]
 [0.98818123 0.00143028 0.00281574 0.01243097 0.00199468]]
----------------------------
step :  10
loss :  0.032737054
accuarcy :  1.0
result :  [[0.01343621 0.0040397  0.00262886 0.98461354 0.00354611]
 [0.98818123 0.00143028 0.00281574 0.01243097 0.00199468]]
----------------------------
step :  20
loss :  0.030769277
accuarcy :  1.0
result :  [[0.01343621 0.0040397  0.00262886 0.98461354 0.00354611]
 [0.98818123 0.00143028 0.00281574 0.01243097 0.00199468]]
----------------------------
step :  30
loss :  0.02901085
accuarcy :  1.0
result :  [[0.01343621 0.0040397  0.00262886 0.98461354 0.00354611]
 [0.98818123 0.00143028 0.00281574 0.01243097 0.00199468]]
----------------------------
step :  40
loss :  0.027430257
accuarcy :  1.0
result :  [[0.01343621 0.0040397  0.00262886 